In [13]:
import pandas as pd
import json
import numpy as np

In [28]:
friend_data = pd.read_csv('grupee_data/friends.csv')
print(friend_data.shape)

with open('grupee_data/preferences.json') as file:
    preferences = json.load(file)
print(preferences['0'])

with open('grupee_data/n_concerts.txt') as file:
    n_concerts = file.read().split('\n')[1:-1]
    
    concerts = np.array([(x.split(':')[0], x.split(':')[1]) for x in n_concerts])

    file.close()


(55483, 1)
000000000000000000000000000000000001000000000101000000000000010001100100000000000000
(84, 2)
